In [27]:
from z3 import *
import itertools

import time
import os

from sat_utils import plot_result, return_coods, optimize

os.makedirs('outputs', exist_ok=True)

In [28]:
def at_least_one(bool_vars):
    return Or(bool_vars)

def at_most_one(bool_vars):
    return [Not(And(pair[0], pair[1])) for pair in itertools.combinations(bool_vars, 2)]

In [45]:
def base_model_order(s,rects,px,py,under,left,W,H):
  #non overlapping condition
  for (i,j) in itertools.combinations(range(len(rects)),2):
    s.add(Or(left[i][j],left[j][i],under[i][j],under[j][i]))

  #implicit domains
  for r in range(len(rects)):
    for xi in range(W-rects[r][0],W):
      s.add(px[r][xi])
    for yi in range(H-rects[r][1],H):
      s.add(py[r][yi])
  #order encoding
  for r in range(len(rects)):
    for e in range(W-1):
      s.add(Or(Not(px[r][e]),px[r][e+1]))
    for f in range(H-1):
      s.add(Or(Not(py[r][f]),py[r][f+1]))



  #meaning for left and under
  for ri in range(len(rects)):
    for rj in range(len(rects)):
      if ri != rj :
        #print((ri,rj))
        s.add(Or(
            Not(left[ri][rj]),
            Not(px[rj][rects[ri][0]-1])
        )) # left(ri,rj) -> xj > wi, lower bound for xj
        for e in range(0,W-rects[ri][0]):
          s.add(Or(
              Not(left[ri][rj]),
              px[ri][e],
              Not(px[rj][e+rects[ri][0]])
          )) #for any e: left(ri,rj) -> (xi <e v ! xj<e+wi)

        s.add(Or(
            Not(under[ri][rj]),
            Not(py[rj][rects[ri][1]-1])
        )) #under(ri,rj)-> yj > hi, lower bound for yj

        for f in range(0,H-rects[ri][1]):
          s.add(Or(
              Not(under[ri][rj]),
              py[ri][f],
              Not(py[rj][f+rects[ri][1]])
        )) #for any f, under(ri,rj) -> (yj <= f+hi ->yi <= f)
  return s

In [46]:
def solve(rects,W,opt=False):
  start = time.time()
  solved= False
  Hmin = int(totarea/W)
  Hmax = int(2*max(max(rects, key=lambda p:p[0]*p[1])[1],totarea/W))+1
  ul = Hmax
  ll = Hmin
  H = ll
  while (ll < ul):
    if (time.time() - start > 300):
      print("out of time")
      return False, (300,300), (0,0,0)
    #print(f"Solving for H={H}")
    px = [[Bool(f"px{r+1}_{x}") for x in range(W)] for r in range(len(rects))]
    py = [[Bool(f"py{r+1}_{y}") for y in range(H)] for r in range(len(rects))]
    under = [[Bool(f"r{ri+1}_under_r{rj+1}") if ri != rj else 0 for rj in range(len(rects)) ] for ri in range(len(rects))]
    left = [[Bool(f"r{ri+1}_leftof_r{rj+1}")  if rj!=ri else 0 for rj in range(len(rects))] for ri in range(len(rects))]
    s = Solver()
    s.set("timeout", 300*1000)

    s = base_model_order(s,rects,px,py,under,left,W,H)
    if opt == True :
      s = optimize(s,rects,px,py,under,left,W,H)

    checking_from = time.time()
    if s.check() == z3.sat :
      ul = H
    else:
      ll = H+1
    H =  int((ll+ul)/2)
    checked_in = time.time() - checking_from
  m = s.model()
  end=time.time()
  elapsed = end-start
  return True, (elapsed,checked_in), (m,return_coods(m,len(rects),px,py),H)

In [ ]:
instances = os.listdir('instances')
for i, filename in enumerate(instances):
  print(f"instance:{filename}")
  with open('instances/'+filename, "r") as file1:
    FileasList = file1.readlines()
  W=int(FileasList[0])
  N=int(FileasList[1])
  rects=[]
  results=[]
  totarea=0
  for n in range(2,2+N):
    values = [int(s) for s in FileasList[n].split() if s.isdigit()]
    w =values[0]
    h = values[1]
    totarea += h*w
    rects.append([w,h])
    results.append([w,h])
  solved,(t_tot,t_solve),(m,c,H) = solve(rects,W, opt=False)
  if solved:
    for r in range(len(rects)):
      results[r].append(c[r][0])
      results[r].append(c[r][1])
    fileout = f"outputs/out-{i}.txt"
    with open(fileout, "w") as file2:
      print(i)
      file2.write(f"{W}\n")
      file2.write(f"{N}\n")
      for i in range(len(results)):
       file2.write(f"{results[i]}\n")
    plot_result(results,W,H)